<a href="https://colab.research.google.com/github/revanthmadasu/machine-learning/blob/master/Lab_6_Template(1)_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

   # Lab 6 - Template

The template is given as a starting point to help you get up to speed with loading file and understanding data.  
Use the template and answer each question in the appropriate cell.

Submit:
- eLearning

Naming: First-name_Last_name_ID_Lab_6.ipynb

## Dataset description

The dataset for Lab-6 is sample of the SBA dataset posted on Kaggle. 

The dataset is from the U.S. Small Business Administration (SBA)
The U.S. SBA was founded in 1953 on the principle of promoting and assisting small enterprises in the U.S. credit market 
(SBA Overview and History, US Small Business Administration (2015)). Small businesses have been a primary source of job creation in the United States; therefore, fostering small business formation and growth has social benefits by creating job opportunities and reducing unemployment.
There have been many success stories of start-ups receiving SBA loan guarantees such as FedEx and Apple Computer. 
However, there have also been stories of small businesses and/or start-ups that have defaulted on their SBA-guaranteed loans.

More info on the original dataset: https://www.kaggle.com/mirbektoktogaraev/should-this-loan-be-approved-or-denied


## Preparation

Load same data into H2O-3 dataframe.  
Describe imported dataframe.

In [1]:
! apt-get install default-jre
!java -version

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-jre is already the newest version (2:1.11-72).
default-jre set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.
openjdk version "11.0.18" 2023-01-17
OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1)
OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)


In [2]:
! pip install h2o

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.7/177.7 MB 4.4 MB/s eta 0:00:00


In [3]:
import h2o
try:
    h2o.cluster().shutdown()
except:
    pass 

In [4]:
# Adjust as per limits on your PC
# Limit to 8 threads and 8GB memory
h2o.init(nthreads=8, max_mem_size=8)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.18" 2023-01-17; OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1); OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.8/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpceomigsc
  JVM stdout: /tmp/tmpceomigsc/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpceomigsc/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,09 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.1
H2O_cluster_version_age:,23 days
H2O_cluster_name:,H2O_from_python_unknownUser_5j66z5
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,8 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [5]:
df_h = h2o.import_file('./data/SBA_loans_lab_6.zip')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
print(df_h.columns)

In [ ]:
df_h.describe()

### Set attributes for the categorical  variables - make non-numerical columns "factor" (categorical)

We will be using H2O-3 DRF model. The model can handle categorical variable and missing values.  
We will need to **mark** categorical variables with `asfactor` method

In [6]:
# Choose which columns to encode
cat_columns = ["City","State","Bank","BankState", "NewExist", "RevLineCr","LowDoc","Zip"]
encoded_columns = cat_columns
response = "Defaulted"

df_h[encoded_columns+[response]] = df_h[encoded_columns+[response]].asfactor()

In [ ]:
df_h.describe()

## Split to train/test/validation

In [7]:
train,test,valid = df_h.split_frame(ratios=[.7, .15], seed=123)

In [ ]:
test

In [ ]:
train

In [ ]:
valid

## Question 1 

Use H2O-3 to train Distributed Random Forest model with default parameters.  
Documentation: https://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/drf.html#defining-a-drf-model

The solution for the problem is provided.
You need to understand the command and the output in order to proceed to the next question.

In [11]:
print(train.columns)

['City', 'State', 'Zip', 'Bank', 'BankState', 'NAICS', 'Term', 'NoEmp', 'NewExist', 'CreateJob', 'RetainedJob', 'FranchiseCode', 'UrbanRural', 'RevLineCr', 'LowDoc', 'DisbursementGross', 'BalanceGross', 'GrAppv', 'SBA_Appv', 'Defaulted']


In [8]:
from h2o.estimators import H2ORandomForestEstimator

predictors = train.columns
predictors.remove("Defaulted")
print("Predictor columns:", predictors)    
response_col = "Defaulted"

drf = H2ORandomForestEstimator(nfolds=0,seed=123)

drf.train(predictors, response_col, training_frame= train)
model_summary = drf.summary()

Predictor columns: ['City', 'State', 'Zip', 'Bank', 'BankState', 'NAICS', 'Term', 'NoEmp', 'NewExist', 'CreateJob', 'RetainedJob', 'FranchiseCode', 'UrbanRural', 'RevLineCr', 'LowDoc', 'DisbursementGross', 'BalanceGross', 'GrAppv', 'SBA_Appv']
drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


In [46]:
model_summary

,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,50.0,50.0,5293375.0,20.0,20.0,20.0,5725.0,7517.0,6730.1


### Model performance metrics on Test dataset

Alternative method to calculate metrics is to use `h2o.make_metrics(pred, actual)` function:   
https://docs.h2o.ai/h2o/latest-stable/h2o-docs/performance-and-prediction.html#computing-model-metrics-from-general-predictions

In [9]:
print("Best accuracy threshold:",drf.model_performance(test).accuracy()[0][0],"\n",
      " Accuracy:",
      drf.model_performance(test).accuracy()[0][1])
print("Best F1 threshold:",drf.model_performance(test).F1()[0][0],"\n",
      " F1:",
      drf.model_performance(test).F1()[0][1])
print("Model AUC:", drf.model_performance(test).auc())
print("Model AUCPR:", drf.model_performance(test).aucpr())
drf.model_performance(test).confusion_matrix()

Best accuracy threshold: 0.4525676262615756 
  Accuracy: 0.9347955611826916
Best F1 threshold: 0.34488910898743635 
  F1: 0.8054782794778516
Model AUC: 0.956193485901203
Model AUCPR: 0.8734735239591254


,0,1,Error,Rate
0,21272.0,871.0,0.0393,(871.0/22143.0)
1,947.0,3764.0,0.201,(947.0/4711.0)
Total,22219.0,4635.0,0.0677,(1818.0/26854.0)


In [ ]:
from h2o import make_metrics as H2OMakeMetrics
pred = drf.predict(test)[2]
print(pred)
actual = test["Defaulted"]
H2OMakeMetrics(pred, actual)


## Question 2

You are asked to tune the model from question #1. You should review full parameter list for the DRF model documentation: https://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/drf.html#defining-a-drf-model

Pay special attention to the following parameters:
- balance_classes
- ntrees
- max_depth
- early stopping params: stopping_rounds, stopping_metric, stopping_tolerance
- mtries
- sample_rate

Alternative to search for the `ntrees` parameter is to set it to high number, and use early stopping to find optimal number of trees. (We will do it in the next question)

Find best parameters using H2O GridSearch for the following search space.
Modify the example code below as needed to make it work.  
You can experiment with additional options/parameters to see if you can get a better model.
```
# DRF hyperparameters
from h2o.grid.grid_search import H2OGridSearch

# DRF hyperparameters
drf_params2  = {'ntrees': [25,50,100],
               'max_depth': [10,15,20,25]}

# Train and validate a cartesian grid of DRFs
drf_grid2  = H2OGridSearch(model=H2ORandomForestEstimator,
                          grid_id='drf_grid2',
                          hyper_params=drf_params2)

drf_grid2.train(x=predictors,
               y=response_col,
               nfolds=0,
               training_frame=train,
               validation_frame=valid,
               seed=123)
```

You can follow the progress of the Grid search job by using H2O Flow, the URL is shown right after `h2o.init()`.
Answer following questions:
- What is the best set of parameters for the DRF model using AUCPR metric?
- What is the best model performance (AUCPR) on a validation dataset?
- What is the best model performance (AUCPR) on a test dataset?
- Produce Confusion Matrix using best F1 probability threshold

In [ ]:
# DRF hyperparameters
from h2o.grid.grid_search import H2OGridSearch

# DRF hyperparameters
# drf_params2  = {'ntrees': [25,50,75, 100],
#                'max_depth': [15,20,25],
#                'sample_rate': [i * 0.1 for i in range(5, 11)],
#                'stopping_tolerance': [0, 0.003, 0.0005],
#                'stopping_rounds': [0,3,5],
#                'stopping_metric': ['misclassification', 'logless', 'custom'],
#                'mtries': [-1, -2], 
#                'balance_classes': [True, False]}
# drf_params2  = {'ntrees': [25,50,75, 100],
#                'max_depth': [15,20,25],
#                'sample_rate': [i * 0.1 for i in range(5, 7)],
#                'stopping_tolerance': [0, 0.003, 0.0005],
#                'stopping_rounds': [0,3,5],
#                'stopping_metric': ['misclassification', 'logless', 'custom'],
#                'mtries': [-1, -2], 
#                'balance_classes': [True, False]}
drf_params3  = {'ntrees': [30],
                'sample_rate': [0.5],
                'stopping_tolerance': [0.003],
                'stopping_rounds': [0],
                'stopping_metric': ['logloss'],
                'mtries': [-1], 
               'balance_classes': [True]}
# Train and validate a cartesian grid of DRFs
drf_grid3  = H2OGridSearch(model=H2ORandomForestEstimator,
                          grid_id='drf_grid3',
                          hyper_params=drf_params3)
drf_grid3.train(x=predictors,
               y=response_col,
               nfolds=0,
               training_frame=train,
               validation_frame=valid,
               seed=123)

In [ ]:
# DRF hyperparameters
from h2o.grid.grid_search import H2OGridSearch

# DRF hyperparameters
# drf_params2  = {'ntrees': [25,50,75, 100],
#                'max_depth': [15,20,25],
#                'sample_rate': [i * 0.1 for i in range(5, 11)],
#                'stopping_tolerance': [0, 0.003, 0.0005],
#                'stopping_rounds': [0,3,5],
#                'stopping_metric': ['misclassification', 'logless', 'custom'],
#                'mtries': [-1, -2], 
#                'balance_classes': [True, False]}
# drf_params2  = {'ntrees': [25,50,75, 100],
#                'max_depth': [15,20,25],
#                'sample_rate': [i * 0.1 for i in range(5, 7)],
#                'stopping_tolerance': [0, 0.003, 0.0005],
#                'stopping_rounds': [0,3,5],
#                'stopping_metric': ['misclassification', 'logless', 'custom'],
#                'mtries': [-1, -2], 
#                'balance_classes': [True, False]}
drf_params2  = {'ntrees': [30,60],
                'sample_rate': [0.5, 0.7],
                'stopping_tolerance': [0, 0.003],
                'stopping_rounds': [0,3],
                'stopping_metric': ['logloss'],
                'mtries': [-1, -2], 
               'balance_classes': [True, False]}
# Train and validate a cartesian grid of DRFs
drf_grid2  = H2OGridSearch(model=H2ORandomForestEstimator,
                          grid_id='drf_grid2',
                          hyper_params=drf_params2)
drf_grid2.train(x=predictors,
               y=response_col,
               nfolds=0,
               training_frame=train,
               validation_frame=valid,
               seed=123)

drf Grid Build progress: |██

In [63]:
drf_gridperf1 = drf_grid2.get_grid(sort_by='auc', decreasing=True)
drf_gridperf1

,max_depth,ntrees,model_ids,auc
,15.0,75.0,drf_grid2_model_41,0.9654584
,25.0,75.0,drf_grid2_model_45,0.9648476
,25.0,75.0,drf_grid2_model_44,0.9647965
,15.0,50.0,drf_grid2_model_29,0.9647276
,20.0,75.0,drf_grid2_model_43,0.9643301
,25.0,50.0,drf_grid2_model_32,0.9643144
,20.0,60.0,drf_grid2_model_125,0.9642226
,20.0,60.0,drf_grid2_model_157,0.9642226
,20.0,60.0,drf_grid2_model_61,0.9642226
,20.0,60.0,drf_grid2_model_93,0.9642226


In [59]:
drf_gridperf1.columns['max_depth']

AttributeError: ignored

In [57]:
best_drf = drf_gridperf1.models[0]
best_drf_perf1 = best_drf.model_performance(test)

best_drf_perf1.auc()

0.9661753751998513

## Question 3

You are asked to tune the model from question #1. You should review full parameter list for the DRF model documentation: https://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/drf.html#defining-a-drf-model

Tune model using early stopping criteria by specifying `ntrees`=1000 and setting the parameters required for early stopping. You will need to provide Validation data-frame for early stopping to work correctly.


Alternative to find optimal setting for `ntrees` (number of trees in RF) parameter is to set it to high number, and use early stopping to find optimal number of trees.

Find best parameters using H2O GridSearch for the following search space.
You should experiment with additional options/parameters to see if you can train better models.
The goal is to get a better model than in Q2. Add at least one additional hyper-parameter to tune.
Minimum parameters to tune:
```
drf_params  = {'max_depth': [10,15, 25]}
```
You should review early stopping parameters to be added to the `train` method:
- stopping_rounds
- stopping_metric
- stopping_tolerance

               
Answer following questions:
- What is the best set of parameters for the DRF model using AUCPR metric?
- What is the best model performance (AUCPR) on a validation dataset?
- What is the best model performance (AUCPR) on a test dataset?
- Produce Confusion Matrix using best F1 probability threshold

## Question 4

Use best model parameters from the GridSearch in question #3. Train new model as following:
- Train model on a combined dataset: train+validation
- Don't use cross-validation - `nfolds=0`
- Evaluate trained model on test dataset
- Compare AUCPR (test dataset) metrics from the question #2 and #3

# Question 5

Review following documentation section first: https://docs.h2o.ai/h2o/latest-stable/h2o-docs/cross-validation.html

- Train DRF model with the parameters found in the Question 3.
- Use full data to train the model.
- Use cross-validation to produce reliable metrics for the model performance
- Produce model metrics using cross-validation 

**Note**: understand why the model training takes significantly longer than before we used cross-validation